In [2]:
import pandas as pd

In [273]:
path_to_file = "/Users/aidanairuser/Desktop/arbres_via_gg.csv"

trees = pd.read_csv(path_to_file, encoding = 'utf-8')


In [274]:
trees.shape

(203530, 17)

In [275]:
trees.head(5)

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,COMPLEMENTADRESSE,NUMERO,LIEU / ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,VARIETEOUCULTIVAR,CIRCONFERENCEENCM,HAUTEUR (m),STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,167154,Arbre,CIMETIERE,SEINE-SAINT-DENIS,NaN,NaN,CIMETIERE DE PANTIN / AVENUE DES SOPHORAS / DI...,A08600083004,Sophora,Sophora,japonica,NaN,0,0,NaN,NaN,"48.9043605622, 2.41027078205"
1,168776,Arbre,CIMETIERE,PARIS 19E ARRDT,NaN,NaN,CIMETIERE DE LA VILLETTE / CHEMIN DE L'EST / D...,A157000F5010,Tilleul,Tilia,cordata,NaN,60,0,NaN,NaN,"48.8849483852, 2.3899845825"
2,186609,Arbre,DASCO,PARIS 18E ARRDT,NaN,NaN,ECOLE ELEMENTAIRE / 9 RUE RICHOMME,111201005,Platane,Platanus,x hispanica,NaN,77,10,JA,0.0,"48.8861703304, 2.35264413443"
3,190877,Arbre,DFPE,PARIS 15E ARRDT,NaN,NaN,CRECHE COLLECTIVE / 73 AVENUE FELIX FAURE,22630006,Platane,Platanus,x hispanica,NaN,150,16,A,0.0,"48.8398807772, 2.28610401896"
4,194855,Arbre,PERIPHERIQUE,PARIS 16E ARRDT,NaN,NaN,TALUS N°16-01,91601008,Peuplier,Populus,alba,Pyramidalis',37,6,J,0.0,"48.8764594303, 2.27959392233"


In [276]:
# renommer certaines colonnes pour plus de facilité

trees.rename(columns={'LIEU / ADRESSE':'ADRESSE', 'HAUTEUR (m)':'HAUTEURENM'}, inplace=True)



## Infos manquantes

In [277]:
# combien de types de 'circonférence' a-t-on?

pd.set_option('display.max_rows', 2000)

trees.CIRCONFERENCEENCM.value_counts()

# 24K lignes / arbres n'ont pas de circonference!!


0         24234
20        10818
70         6631
60         6369
50         6257
80         6103
90         5782
40         5769
30         5746
100        5452
25         5354
45         5038
75         4952
35         4862
65         4691
55         4631
110        4622
85         4313
120        3995
95         3777
130        3489
105        3478
115        2985
140        2947
150        2723
125        2571
135        2248
160        2244
145        1982
170        1888
180        1641
155        1561
10         1413
165        1358
190        1337
175        1245
200        1225
185         926
210         889
15          860
195         813
220         754
205         617
230         597
215         524
240         466
225         385
250         377
62          367
63          362
67          359
72          355
42          347
48          345
66          335
73          334
68          333
58          330
57          328
52          327
78          323
53          323
82      

In [279]:
# combien de lignes ont une circonference > 5m?, 117 paraît-il.... !!!!

trees[trees.CIRCONFERENCEENCM > 500].count()

IDBASE                117
TYPEEMPLACEMENT       117
DOMANIALITE           117
ARRONDISSEMENT        117
COMPLEMENTADRESSE      42
NUMERO                  0
ADRESSE               117
IDEMPLACEMENT         117
LIBELLEFRANCAIS       116
GENRE                 117
ESPECE                116
VARIETEOUCULTIVAR      11
CIRCONFERENCEENCM     117
HAUTEURENM            117
STADEDEVELOPPEMENT    108
REMARQUABLE           111
geo_point_2d          117
dtype: int64

In [281]:
# combien de genres différents?

trees.GENRE.value_counts().count()

175

In [282]:
# combien de espèces différents?

trees.ESPECE.value_counts().count()

537

### On décide de remplacer les valeurs abérrents avec les moyennes des genres (même si une moyenne par espèce est peut-être plus juste. A revoir.

In [283]:
"""
créer un df sans les circonferences 0s et les circonferences plus grand que 5m, afin de trouver des moyennes crédibles
"""

trees_propre_circon = trees[(trees['CIRCONFERENCEENCM'] != 0 ) & (trees['CIRCONFERENCEENCM'] <= 500)]

In [284]:
# créer un vrai nouveau DF depuis notre selection

clean_df = pd.DataFrame(trees_propre_circon)

In [286]:
""" Je vérifie : le nouveau df a bien le bon nombre de lignes, cad toutes sauf les 0s (24234) et les > 5m (117)"""

203_530 - 24_234 - 117

179179

In [285]:
# oui, le nouveau df a bien le nombre de lignes attendu

clean_df.shape

(179179, 17)

In [287]:
# sors-moi la circonference moyenne par genre, arrondi

no_dec = clean_df.groupby('GENRE').CIRCONFERENCEENCM.mean().astype(int)

# regardons les, en ordre - on constate qu'il n'y a pas de moyenne 0

no_dec.sort_values()

GENRE
Luma                   5
Asimina                7
Acca                  10
Rhamnus               10
Sapindus              10
Genista               10
Callistemon           10
Fremontodendron       10
Caragana              17
Buddleja              20
Heptacodium           20
Carica                20
Syringa               20
Aronia                20
Alangium              23
Punica                24
Nyssa                 24
x Chitalpa            24
Idesia                25
Lagerstroemia         25
Kalopanax             25
Ziziphus              25
Amelanchier           26
Cydonia               27
Hibiscus              27
Cinnamomum            28
Mespilus              29
Laburnocytisus        30
Aralia                30
Clerodendrum          30
Castanea              30
Hovenia               30
Poliothyrsis          31
Chimonanthus          32
Photinia              33
Malus                 33
Albizia               33
Enkianthus            33
Viburnum              33
Parrotia           

In [288]:
# créer un dictionnaire de genre-circonference_moyenne à partir de notre sélection

circ_moy = no_dec.to_daict()

type (circ_moy)

dict

In [289]:
# vérifier qu'en rentrant un clé 'genre', on nous rend une valeur 'moyenne'

circ_moy["Aronia"]

20

## Certains genres n'ont aucun info sur les circonferences donc ne peuvent pas fournir de moyenne. 

In [290]:
# il y a 175 genres en tout

trees.GENRE.nunique()

175

In [291]:
# ATTENTION, sur les 175 genres combien de moyennes a-t-on dans notre dictionnaire?

len (no_dec)

# que 158, alors il y a 17 genres qui ont probablement 0 comme moyenne

158

In [306]:
# on crée une liste de 175 genres. Il y a 176 éléments?

all_genres = trees.GENRE.unique()
genre_list = all_genres.tolist()

len(genre_list)

176

In [296]:
# on crée une liste depuis les clés du dictionnaire

no_dec_key_list = no_dec.keys().tolist()

no_dec_key_list

['Abelia',
 'Abies',
 'Acacia',
 'Acca',
 'Acer',
 'Aesculus',
 'Ailanthus',
 'Alangium',
 'Albizia',
 'Alnus',
 'Amelanchier',
 'Aralia',
 'Araucaria',
 'Arbutus',
 'Aronia',
 'Asimina',
 'Betula',
 'Brachychiton',
 'Broussonetia',
 'Buddleja',
 'Buxus',
 'Callistemon',
 'Calocedrus',
 'Caragana',
 'Carica',
 'Carpinus',
 'Carya',
 'Castanea',
 'Catalpa',
 'Cedrus',
 'Celtis',
 'Cephalotaxus',
 'Cercidiphyllum',
 'Cercis',
 'Chamaecyparis',
 'Chimonanthus',
 'Cinnamomum',
 'Cladrastis',
 'Clerodendrum',
 'Cordyline',
 'Cornus',
 'Corylus',
 'Cotinus',
 'Cotoneaster',
 'Crataegus',
 'Cryptomeria',
 'Cunninghamia',
 'Cupressus',
 'Cydonia',
 'Davidia',
 'Diospyros',
 'Ehretia',
 'Elaeagnus',
 'Enkianthus',
 'Eriobotrya',
 'Eucalyptus',
 'Eucommia',
 'Euonymus',
 'Euscaphis',
 'Fagus',
 'Ficus',
 'Firmiana',
 'Fontanesia',
 'Fraxinus',
 'Fremontodendron',
 'Genista',
 'Ginkgo',
 'Gleditsia',
 'Gymnocladus',
 'Heptacodium',
 'Hibiscus',
 'Hippophae',
 'Hovenia',
 'Idesia',
 'Ilex',
 'Jugl

### On revient sur le df de base, et on vire les 17 genres qui ne sont pas dans le dictionnaire (parce qu'ils n'ont pas d'info du tout en circonference)

In [321]:
# on fait une df / selection où les lignes sont retenues uniquement si le genre paraît dans le dictionnaire

ok_df = trees[trees.GENRE.isin(no_dec_key_list)]

# on crée un nouveau df depuis cette selection

df_no_zeros = pd.DataFrame(ok_df)

In [322]:
# notre selection a maintenant combien de genres?

df_no_zeros.GENRE.nunique()

# le même nombre que dans le dico. C'est bon

158

In [323]:
# et il a combien de lignes?

df_no_zeros.shape

(203492, 17)

## On fait un boucle pour trouver une circonférence abérrante, identifier le genre, retrouver une moyenne pour ce genre, et remplace la circonférence abérrante avec la moyenne

In [324]:
# circ_moy est notre dictionnaire de moyennes pour chaque genre
# liste_a_virer est notre liste des genres qui n'ont pas d'infos du tout pour fournir une moyenne (et qui sont donc
# à virer)

row = 0
length = len(df_no_zeros)
replace_counter = 0

for r in range (length):

    if df_no_zeros.iat[row, 12] == 0 or df_no_zeros.iat[row, 12] >= 500:
        genre = df_no_zeros.iat[row, 9]
        val = circ_moy[genre]
        df_no_zeros.iat[row, 12] = val
        row = row + 1
        replace_counter = replace_counter + 1
    
    else:
        row = row + 1

print ("total nb of circonferences replaced:", replace_counter)

total nb of circonferences replaced: 24317


In [325]:
# on regarde le début du df maintenant: la valuer 0 de la première ligne (circonférence) a bien été remplacé par
# 88, la moyenne pour le genre 

df_no_zeros.head(5)

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,COMPLEMENTADRESSE,NUMERO,ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,VARIETEOUCULTIVAR,CIRCONFERENCEENCM,HAUTEURENM,STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d
0,167154,Arbre,CIMETIERE,SEINE-SAINT-DENIS,NaN,NaN,CIMETIERE DE PANTIN / AVENUE DES SOPHORAS / DI...,A08600083004,Sophora,Sophora,japonica,NaN,88,0,NaN,NaN,"48.9043605622, 2.41027078205"
1,168776,Arbre,CIMETIERE,PARIS 19E ARRDT,NaN,NaN,CIMETIERE DE LA VILLETTE / CHEMIN DE L'EST / D...,A157000F5010,Tilleul,Tilia,cordata,NaN,60,0,NaN,NaN,"48.8849483852, 2.3899845825"
2,186609,Arbre,DASCO,PARIS 18E ARRDT,NaN,NaN,ECOLE ELEMENTAIRE / 9 RUE RICHOMME,111201005,Platane,Platanus,x hispanica,NaN,77,10,JA,0.0,"48.8861703304, 2.35264413443"
3,190877,Arbre,DFPE,PARIS 15E ARRDT,NaN,NaN,CRECHE COLLECTIVE / 73 AVENUE FELIX FAURE,22630006,Platane,Platanus,x hispanica,NaN,150,16,A,0.0,"48.8398807772, 2.28610401896"
4,194855,Arbre,PERIPHERIQUE,PARIS 16E ARRDT,NaN,NaN,TALUS N°16-01,91601008,Peuplier,Populus,alba,Pyramidalis',37,6,J,0.0,"48.8764594303, 2.27959392233"


In [326]:
# vérifier qu'il n'y a plus de lignes avec une circonférence 0

df_no_zeros[df_no_zeros.CIRCONFERENCEENCM ==0]

# il n'y en a plus!

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,COMPLEMENTADRESSE,NUMERO,ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,VARIETEOUCULTIVAR,CIRCONFERENCEENCM,HAUTEURENM,STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d


In [329]:
# vérifier qu'il n'y a plus de lignes avec une circonférence 0

df_no_zeros[df_no_zeros.CIRCONFERENCEENCM > 500]

# il n'y en a plus!

,IDBASE,TYPEEMPLACEMENT,DOMANIALITE,ARRONDISSEMENT,COMPLEMENTADRESSE,NUMERO,ADRESSE,IDEMPLACEMENT,LIBELLEFRANCAIS,GENRE,ESPECE,VARIETEOUCULTIVAR,CIRCONFERENCEENCM,HAUTEURENM,STADEDEVELOPPEMENT,REMARQUABLE,geo_point_2d


## finalement j'ai pas utilisé

In [307]:
 
# on sort une liste des genres qui ne paraissent pas dans le dictionnaire, et qui donc n'ont probablement pas d'info    
    
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 
  
li1 = genre_list 
li2 = no_dec_key_list

liste_a_virer = (Diff(li1, li2)) 
print (liste_a_virer)



[nan, 'Philadelphus', 'Stewartia', 'x Crataemespilus', 'x Sorbopyrus', 'Chionanthus', 'Parrotiopsis', 'Sciadopitys', 'Hemiptelea', 'Xanthoceras', 'Garrya', 'Pittosporum', 'Podocarpus', 'Brahea', 'Washingtonia', 'Rhododendron', 'Lonicera', 'Cudrania']


In [308]:
type(liste_a_virer)

list